In [1]:
# import dependencies
import sqlalchemy
from sqlalchemy import Column, Integer, String, Numeric, Float, func
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import pandas as pd
import numpy as np
import requests, time, json
import googlemaps, gmaps
from API_KEY import API_KEY
from pprint import pprint


##### (Data obtained from Kaggle.com) 
READ in scrubbed New York brewery csv file after correcting city, state and zip code for 5 locations in Excel.
Created api_address column manually in the event is would be needed to get geocodes for cross reference

In [2]:
brewfile = "nybrews_4api_file2.csv"
newbrew_df = pd.read_csv(brewfile)
newbrew_df.head()

,index2,brewery_name,address,city,state,zip_code,api_address
0,0,Typhoon Brewery,22 E. 54th St. (@ Madison),New York,NY,10022,"22+E.+54th+St.+(@+Madison),New+York,NY,10022"
1,1,The Riverosa Company,101 W. 75th St. Suite 5B,New York,NY,10023,"101+W.+75th+St.+Suite+5B,New+York,NY,10023"
2,2,Spring Street Brewing Company,113 University Place - 11th Floor,New York,NY,10003,"113+University+Place+-+11th+Floor,New+York,NY,..."
3,3,Rheingold Brewing Company,130 West 42nd St.,New York,NY,10036,"130+West+42nd+St.,New+York,NY,10036"
4,4,Premier Beverages LLC,590 Madison Ave. 21st Fl.,New York,NY,10022,"590+Madison+Ave.+21st+Fl.,New+York,NY,10022"


In [3]:
# dropping index2 and api_address columns given it was determined they are no longer needed for this exercise
newbrew_df = newbrew_df.drop(['index2', 'api_address'], axis=1)
newbrew_df.head()

,brewery_name,address,city,state,zip_code
0,Typhoon Brewery,22 E. 54th St. (@ Madison),New York,NY,10022
1,The Riverosa Company,101 W. 75th St. Suite 5B,New York,NY,10023
2,Spring Street Brewing Company,113 University Place - 11th Floor,New York,NY,10003
3,Rheingold Brewing Company,130 West 42nd St.,New York,NY,10036
4,Premier Beverages LLC,590 Madison Ave. 21st Fl.,New York,NY,10022


##### use googlemaps below to get geo coordinates for the 16 brewery addresses on our BREWERY TOUR itinerary 

In [5]:
# API_KEY = 'INSERT_MAPS_API_KEY_HERE'
gmaps = googlemaps.Client(key=API_KEY)

In [6]:
def get_coordinates(address):
    city = 'New York, US'
    geocode_result = gmaps.geocode(str(address) +' '+ city)
    if len(geocode_result) > 0:
        return list(geocode_result[0]['geometry']['location'].values())
    else:
        return [np.NaN, np.NaN]

coordinates = newbrew_df['address'].apply(lambda x: pd.Series(get_coordinates(x), index=['LATITUDE', 'LONGITUDE']))
newbrew_df5 = pd.concat([newbrew_df[:], coordinates[:]], axis="columns")

In [7]:
# Assign better column headers
newbrew_df5.columns = ['brewery','address','city','state','zip_code','brew_lat', 'brew_lon']
newbrew_df5

,brewery,address,city,state,zip_code,brew_lat,brew_lon
0,Typhoon Brewery,22 E. 54th St. (@ Madison),New York,NY,10022,40.760342,-73.974326
1,The Riverosa Company,101 W. 75th St. Suite 5B,New York,NY,10023,40.779579,-73.977562
2,Spring Street Brewing Company,113 University Place - 11th Floor,New York,NY,10003,40.734271,-73.992162
3,Rheingold Brewing Company,130 West 42nd St.,New York,NY,10036,40.755123,-73.985463
4,Premier Beverages LLC,590 Madison Ave. 21st Fl.,New York,NY,10022,40.762110,-73.973069
5,New Amsterdam Brewing Co.,257 Park Ave. S.,New York,NY,10010,40.738623,-73.987135
6,Neptune Brewery,448 W. 16th St.,New York,NY,10011,40.742893,-74.006416
7,Nacho Mama's Brewery,42-42 Thompson St.,New York,NY,10013,40.723304,-74.003707
8,Manhattan Brewing Co. Restaurant,40-42 Thompson St.,New York,NY,10013,40.723278,-74.003745
9,John Street Bar & Grill,17 John St.,New York,NY,10038,40.709970,-74.008741


#####  Establish connection with Postgres etlproj_db and engine for query purposes 
##### Transfer brewery dataframe to Postgres to create brewery table in etlproj_db

In [8]:
from postgreslogin import postgres_key

MyPostLogin = postgres_key
rds_connection_string = MyPostLogin
engine = create_engine(f'postgresql://{rds_connection_string}')

In [9]:
newbrew_df5.to_sql(name='ny_brews2', con=engine, if_exists='replace', index=False)
# read back from SQL Postgres for test purposes
pd.read_sql_query('select * from ny_brews2', con=engine).head()


,brewery,address,city,state,zip_code,brew_lat,brew_lon
0,Typhoon Brewery,22 E. 54th St. (@ Madison),New York,NY,10022,40.760342,-73.974326
1,The Riverosa Company,101 W. 75th St. Suite 5B,New York,NY,10023,40.779579,-73.977562
2,Spring Street Brewing Company,113 University Place - 11th Floor,New York,NY,10003,40.734271,-73.992162
3,Rheingold Brewing Company,130 West 42nd St.,New York,NY,10036,40.755123,-73.985463
4,Premier Beverages LLC,590 Madison Ave. 21st Fl.,New York,NY,10022,40.762110,-73.973069


In [10]:
##############################################################################################################

##### AirBnB Information #####

In [12]:
# Read in New York AirBnb CSV file obtained from Kaggle.com
csv_file = "Resources/AB_NYC_2019.csv"
AB_NYC_2019_data_df = pd.read_csv(csv_file)
AB_NYC_2019_data_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [13]:
# drop some unnecessary columns from AirBnB dataframe
AB_NYC_2019_data_df = AB_NYC_2019_data_df.drop(['last_review', 'reviews_per_month', 'number_of_reviews',\
'calculated_host_listings_count', 'neighbourhood', 'neighbourhood_group'], axis=1)
AB_NYC_2019_data_df.head()

,id,name,host_id,host_name,latitude,longitude,room_type,price,minimum_nights,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,40.64749,-73.97237,Private room,149,1,365
1,2595,Skylit Midtown Castle,2845,Jennifer,40.75362,-73.98377,Entire home/apt,225,1,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,40.80902,-73.94190,Private room,150,3,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,40.68514,-73.95976,Entire home/apt,89,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,40.79851,-73.94399,Entire home/apt,80,10,0


In [14]:
# Assign better column headers
AB_NYC_2019_data_df.columns = ['airbnb_id','airbnb_name','host_id','host_name','air_lat', 'air_lon',\
                               'room_type', 'price', 'minimum_nights', 'availability_365']
AB_NYC_2019_data_df

,airbnb_id,airbnb_name,host_id,host_name,air_lat,air_lon,room_type,price,minimum_nights,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,40.64749,-73.97237,Private room,149,1,365
1,2595,Skylit Midtown Castle,2845,Jennifer,40.75362,-73.98377,Entire home/apt,225,1,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,40.80902,-73.94190,Private room,150,3,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,40.68514,-73.95976,Entire home/apt,89,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,40.79851,-73.94399,Entire home/apt,80,10,0
...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,40.67853,-73.94995,Private room,70,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,40.70184,-73.93317,Private room,40,4,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,40.81475,-73.94867,Entire home/apt,115,10,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,40.75751,-73.99112,Shared room,55,1,2


#### Create CSV file for AirBnB data
#### Establish connection with Postgres etlproj_db and engine for query purposes
#### Transfer AirBnB data to Postgres etlproj_db to populate the AirBnB table

In [15]:
# Create CSV file for AirBnB data
AB_NYC_2019_data_df.to_csv('nyabnb.csv')

In [16]:
# Establish connection with Postgres etlproj_db and engine for query purposes
from postgreslogin import postgres_key

MyPostLogin = postgres_key
rds_connection_string = MyPostLogin
engine = create_engine(f'postgresql://{rds_connection_string}')

In [17]:
#Transfer AirBnB data to Postgres etlproj_db to populate the AirBnB table
AB_NYC_2019_data_df.to_sql(name='ny_abnb', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from ny_abnb', con=engine).head()

,airbnb_id,airbnb_name,host_id,host_name,air_lat,air_lon,room_type,price,minimum_nights,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,40.64749,-73.97237,Private room,149,1,365
1,2595,Skylit Midtown Castle,2845,Jennifer,40.75362,-73.98377,Entire home/apt,225,1,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,40.80902,-73.94190,Private room,150,3,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,40.68514,-73.95976,Entire home/apt,89,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,40.79851,-73.94399,Entire home/apt,80,10,0


In [18]:

##############################################################################################################

In [19]:
# Establish connection with Postgres etlproj_db and engine for query purposes
MyPostLogin = postgres_key
rds_connection_string = MyPostLogin
engine = create_engine(f'postgresql://{rds_connection_string}')
session = Session(engine)

In [20]:
# check for defined classes in Postgres etlproj_db
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

[]

In [21]:
# check for table names in Postgres etlproj_db
inspector = inspect(engine)
inspector.get_table_names()


['ny_breweries', 'distance', 'ny_brews2', 'ny_abnb']

In [22]:
# get the column names for Brewery table in Postgres etlproj_db
columns = inspector.get_columns('ny_brews2')
for c in columns:
    print(c['name'], c["type"])


brewery TEXT
address TEXT
city TEXT
state TEXT
zip_code BIGINT
brew_lat DOUBLE PRECISION
brew_lon DOUBLE PRECISION


In [23]:
# # get the column names for AirBnB table in Postgres etlproj_db
columns = inspector.get_columns('ny_abnb')
for c in columns:
    print(c['name'], c["type"])

airbnb_id BIGINT
airbnb_name TEXT
host_id BIGINT
host_name TEXT
air_lat DOUBLE PRECISION
air_lon DOUBLE PRECISION
room_type TEXT
price BIGINT
minimum_nights BIGINT
availability_365 BIGINT


In [24]:
##############################################################################################################

#####  perform a "cross join" of the brewery and airbnb tables, save as crossjoin_df and then perform pandas iterrows() to get a list of just the required columns.

In [26]:
crossjoin_df = pd.read_sql_query('SELECT * FROM ny_brews2 CROSS JOIN ny_abnb', con=engine)

In [27]:
crossjoin_df

,brewery,address,city,state,zip_code,brew_lat,brew_lon,airbnb_id,airbnb_name,host_id,host_name,air_lat,air_lon,room_type,price,minimum_nights,availability_365
0,Typhoon Brewery,22 E. 54th St. (@ Madison),New York,NY,10022,40.760342,-73.974326,2539,Clean & quiet apt home by the park,2787,John,40.64749,-73.97237,Private room,149,1,365
1,The Riverosa Company,101 W. 75th St. Suite 5B,New York,NY,10023,40.779579,-73.977562,2539,Clean & quiet apt home by the park,2787,John,40.64749,-73.97237,Private room,149,1,365
2,Spring Street Brewing Company,113 University Place - 11th Floor,New York,NY,10003,40.734271,-73.992162,2539,Clean & quiet apt home by the park,2787,John,40.64749,-73.97237,Private room,149,1,365
3,Rheingold Brewing Company,130 West 42nd St.,New York,NY,10036,40.755123,-73.985463,2539,Clean & quiet apt home by the park,2787,John,40.64749,-73.97237,Private room,149,1,365
4,Premier Beverages LLC,590 Madison Ave. 21st Fl.,New York,NY,10022,40.762110,-73.973069,2539,Clean & quiet apt home by the park,2787,John,40.64749,-73.97237,Private room,149,1,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831210,Hansens Times Square Brewery,160 W. 42nd St.,New York,NY,10036,40.755584,-73.986483,36487245,Trendy duplex in the very heart of Hell's Kitchen,68119814,Christophe,40.76404,-73.98933,Private room,90,7,23
831211,Commonwealth Brewing Co.,10 Rockefeller Plaza (@48th St.),New York,NY,10020,40.758046,-73.979444,36487245,Trendy duplex in the very heart of Hell's Kitchen,68119814,Christophe,40.76404,-73.98933,Private room,90,7,23
831212,Chelsea Brewing Co.,59 Chelsea Piers,New York,NY,10011,40.745682,-74.008722,36487245,Trendy duplex in the very heart of Hell's Kitchen,68119814,Christophe,40.76404,-73.98933,Private room,90,7,23
831213,Carnegie Hill Brewing Co.,1600 3rd Ave. (@ 90th St.),New York,NY,10028,40.781370,-73.952445,36487245,Trendy duplex in the very heart of Hell's Kitchen,68119814,Christophe,40.76404,-73.98933,Private room,90,7,23


In [ ]:
##############################################################################################################

In [32]:
# Create an empty list and perform iterrows over the crossjoin_df and append to new_list the output of the f-string
new_list = []
for index, row in crossjoin_df.iterrows():
    biglist = f"({row['brewery']}, {row['airbnb_id']}, {row['brew_lat']}, {row['brew_lon']}, {row['air_lat']}, {row['air_lon']})"
    new_list.append(biglist)
print(type(new_list))
new_list

<class 'list'>


['(Typhoon Brewery, 2539, 40.760342, -73.974326, 40.647490000000005, -73.97237)',
 '(The Riverosa Company, 2539, 40.7795789, -73.9775621, 40.647490000000005, -73.97237)',
 '(Spring Street Brewing Company, 2539, 40.7342711, -73.9921624, 40.647490000000005, -73.97237)',
 '(Rheingold Brewing Company, 2539, 40.7551234, -73.98546329999999, 40.647490000000005, -73.97237)',
 '(Premier Beverages LLC, 2539, 40.76211, -73.973069, 40.647490000000005, -73.97237)',
 '(New Amsterdam Brewing Co., 2539, 40.7386232, -73.9871351, 40.647490000000005, -73.97237)',
 '(Neptune Brewery, 2539, 40.74289340000001, -74.0064163, 40.647490000000005, -73.97237)',
 "(Nacho Mama's Brewery, 2539, 40.7233037, -74.00370749999999, 40.647490000000005, -73.97237)",
 '(Manhattan Brewing Co. Restaurant, 2539, 40.7232777, -74.0037451, 40.647490000000005, -73.97237)',
 '(John Street Bar & Grill, 2539, 40.7099702, -74.0087412, 40.647490000000005, -73.97237)',
 '(Heartland Brewery, 2539, 40.73688990000001, -73.990792, 40.6474900

In [33]:
# Create an empty list and perform iterrows over the crossjoin_df and append to Row_list1 variable my_list1
Row_list1 =[] 

for index, row in crossjoin_df.iterrows():
    my_list1 = ({row['brewery']}, {row['airbnb_id']}, {row['brew_lat']}, {row['brew_lon']}, {row['air_lat']}, {row['air_lon']})
          
    # append the list to the final list 
    Row_list1.append(my_list1)
    
# Print the list 
# print(Row_list1)
print(type(Row_list1))
Row_list1

<class 'list'>


[({'Typhoon Brewery'},
  {2539},
  {40.760342},
  {-73.974326},
  {40.647490000000005},
  {-73.97237}),
 ({'The Riverosa Company'},
  {2539},
  {40.7795789},
  {-73.9775621},
  {40.647490000000005},
  {-73.97237}),
 ({'Spring Street Brewing Company'},
  {2539},
  {40.7342711},
  {-73.9921624},
  {40.647490000000005},
  {-73.97237}),
 ({'Rheingold Brewing Company'},
  {2539},
  {40.7551234},
  {-73.98546329999999},
  {40.647490000000005},
  {-73.97237}),
 ({'Premier Beverages LLC'},
  {2539},
  {40.76211},
  {-73.973069},
  {40.647490000000005},
  {-73.97237}),
 ({'New Amsterdam Brewing Co.'},
  {2539},
  {40.7386232},
  {-73.9871351},
  {40.647490000000005},
  {-73.97237}),
 ({'Neptune Brewery'},
  {2539},
  {40.74289340000001},
  {-74.0064163},
  {40.647490000000005},
  {-73.97237}),
 ({"Nacho Mama's Brewery"},
  {2539},
  {40.7233037},
  {-74.00370749999999},
  {40.647490000000005},
  {-73.97237}),
 ({'Manhattan Brewing Co. Restaurant'},
  {2539},
  {40.7232777},
  {-74.0037451},
  {

#### This is the list format that will be used to create the next dataframe "crossjoin_df2"

In [34]:
# Create an empty list and perform iterrows over the crossjoin_df and append to Row_list2 variable my_list2
Row_list2 =[] 
  
# Iterate over each row 
for index, rows in crossjoin_df.iterrows(): 
    # Create list for the current row 
    my_list2 =[rows.brewery, rows.airbnb_id, rows.brew_lat, rows.brew_lon, rows.air_lat, rows.air_lon] 
      
    # append the list to the final list 
    Row_list2.append(my_list2) 

# Print Row_list
print(type(Row_list2))
Row_list2

<class 'list'>


[['Typhoon Brewery',
  2539,
  40.760342,
  -73.974326,
  40.647490000000005,
  -73.97237],
 ['The Riverosa Company',
  2539,
  40.7795789,
  -73.9775621,
  40.647490000000005,
  -73.97237],
 ['Spring Street Brewing Company',
  2539,
  40.7342711,
  -73.9921624,
  40.647490000000005,
  -73.97237],
 ['Rheingold Brewing Company',
  2539,
  40.7551234,
  -73.98546329999999,
  40.647490000000005,
  -73.97237],
 ['Premier Beverages LLC',
  2539,
  40.76211,
  -73.973069,
  40.647490000000005,
  -73.97237],
 ['New Amsterdam Brewing Co.',
  2539,
  40.7386232,
  -73.9871351,
  40.647490000000005,
  -73.97237],
 ['Neptune Brewery',
  2539,
  40.74289340000001,
  -74.0064163,
  40.647490000000005,
  -73.97237],
 ["Nacho Mama's Brewery",
  2539,
  40.7233037,
  -74.00370749999999,
  40.647490000000005,
  -73.97237],
 ['Manhattan Brewing Co. Restaurant',
  2539,
  40.7232777,
  -74.0037451,
  40.647490000000005,
  -73.97237],
 ['John Street Bar & Grill',
  2539,
  40.7099702,
  -74.0087412,
  40.

In [35]:
crossjoin_df2 = pd.DataFrame(Row_list2)

In [36]:
crossjoin_df2

,0,1,2,3,4,5
0,Typhoon Brewery,2539,40.760342,-73.974326,40.64749,-73.97237
1,The Riverosa Company,2539,40.779579,-73.977562,40.64749,-73.97237
2,Spring Street Brewing Company,2539,40.734271,-73.992162,40.64749,-73.97237
3,Rheingold Brewing Company,2539,40.755123,-73.985463,40.64749,-73.97237
4,Premier Beverages LLC,2539,40.762110,-73.973069,40.64749,-73.97237
...,...,...,...,...,...,...
831210,Hansens Times Square Brewery,36487245,40.755584,-73.986483,40.76404,-73.98933
831211,Commonwealth Brewing Co.,36487245,40.758046,-73.979444,40.76404,-73.98933
831212,Chelsea Brewing Co.,36487245,40.745682,-74.008722,40.76404,-73.98933
831213,Carnegie Hill Brewing Co.,36487245,40.781370,-73.952445,40.76404,-73.98933


In [37]:
crossjoin_df2.columns = ['brewery','airbnb_id','brew_lat','brew_lon','air_lat','air_lon']

In [38]:
crossjoin_df2.head()

,brewery,airbnb_id,brew_lat,brew_lon,air_lat,air_lon
0,Typhoon Brewery,2539,40.760342,-73.974326,40.64749,-73.97237
1,The Riverosa Company,2539,40.779579,-73.977562,40.64749,-73.97237
2,Spring Street Brewing Company,2539,40.734271,-73.992162,40.64749,-73.97237
3,Rheingold Brewing Company,2539,40.755123,-73.985463,40.64749,-73.97237
4,Premier Beverages LLC,2539,40.762110,-73.973069,40.64749,-73.97237


In [39]:
#This is code to limit rows to 17 for testing purposes only
# crossjoin_df3 = crossjoin_df2
# crossjoin_df4 = crossjoin_df3.iloc[0:17]

In [40]:
crossjoin_df4 = crossjoin_df2

In [41]:
crossjoin_df4

,brewery,airbnb_id,brew_lat,brew_lon,air_lat,air_lon
0,Typhoon Brewery,2539,40.760342,-73.974326,40.64749,-73.97237
1,The Riverosa Company,2539,40.779579,-73.977562,40.64749,-73.97237
2,Spring Street Brewing Company,2539,40.734271,-73.992162,40.64749,-73.97237
3,Rheingold Brewing Company,2539,40.755123,-73.985463,40.64749,-73.97237
4,Premier Beverages LLC,2539,40.762110,-73.973069,40.64749,-73.97237
...,...,...,...,...,...,...
831210,Hansens Times Square Brewery,36487245,40.755584,-73.986483,40.76404,-73.98933
831211,Commonwealth Brewing Co.,36487245,40.758046,-73.979444,40.76404,-73.98933
831212,Chelsea Brewing Co.,36487245,40.745682,-74.008722,40.76404,-73.98933
831213,Carnegie Hill Brewing Co.,36487245,40.781370,-73.952445,40.76404,-73.98933


###### The following code uses the HAVERSINE function to calculate the distance between the 16 breweries and all the NY AirBnB's 

### ***** This will take 1.5 hrs to run unless you limit the data in the 3 cells above ******

In [42]:
# Use the Haversine Function to calculate the distance between the coordinates of the Brewery and each AirBnB

from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    Miles = 3956 * c
    return Miles


for index, row in crossjoin_df4.iterrows():
        crossjoin_df4.loc[index, 'distance'] = haversine(row[3], row[2], row[5], row[4])
        
crossjoin_df4.head(17)

,brewery,airbnb_id,brew_lat,brew_lon,air_lat,air_lon,distance
0,Typhoon Brewery,2539,40.760342,-73.974326,40.64749,-73.97237,7.792564
1,The Riverosa Company,2539,40.779579,-73.977562,40.64749,-73.97237,9.124155
2,Spring Street Brewing Company,2539,40.734271,-73.992162,40.64749,-73.97237,6.080756
3,Rheingold Brewing Company,2539,40.755123,-73.985463,40.64749,-73.97237,7.463109
4,Premier Beverages LLC,2539,40.762110,-73.973069,40.64749,-73.97237,7.914048
5,New Amsterdam Brewing Co.,2539,40.738623,-73.987135,40.64749,-73.97237,6.339611
6,Neptune Brewery,2539,40.742893,-74.006416,40.64749,-73.97237,6.824011
7,Nacho Mama's Brewery,2539,40.723304,-74.003707,40.64749,-73.97237,5.485689
8,Manhattan Brewing Co. Restaurant,2539,40.723278,-74.003745,40.64749,-73.97237,5.484566
9,John Street Bar & Grill,2539,40.709970,-74.008741,40.64749,-73.97237,4.715642


In [ ]:
########  This cell of code is for verifying any single calculaton of the large file above #############
from math import radians, cos, sin, asin, sqrt 
from haversine import haversine, Unit

def distance(lat1, lat2, lon1, lon2): 
      
    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers is 6371. Use 3956 for miles. Use 20908800 for feet. Use 6969600 for yards.
    r = 3956
       
    # calculate the result 
    return(c * r) 
      
      
# driver code  
lat1 = 40.760342
lat2 = 40.647490000000005
lon1 = -73.974326
lon2 =  -73.97237
print(distance(lat1, lat2, lon1, lon2), "Miles")

In [43]:
# Confirming  Column Headers
crossjoin_df4.columns

Index(['brewery', 'airbnb_id', 'brew_lat', 'brew_lon', 'air_lat', 'air_lon',
       'distance'],
      dtype='object')

In [44]:
# Assign New Column Headers if necessary
crossjoin_df4.columns = ['brewery','airbnb_id','brew_lat','brew_lon','air_lat','air_lon', 'dist']

In [45]:
crossjoin_df4

,brewery,airbnb_id,brew_lat,brew_lon,air_lat,air_lon,dist
0,Typhoon Brewery,2539,40.760342,-73.974326,40.64749,-73.97237,7.792564
1,The Riverosa Company,2539,40.779579,-73.977562,40.64749,-73.97237,9.124155
2,Spring Street Brewing Company,2539,40.734271,-73.992162,40.64749,-73.97237,6.080756
3,Rheingold Brewing Company,2539,40.755123,-73.985463,40.64749,-73.97237,7.463109
4,Premier Beverages LLC,2539,40.762110,-73.973069,40.64749,-73.97237,7.914048
...,...,...,...,...,...,...,...
831210,Hansens Times Square Brewery,36487245,40.755584,-73.986483,40.76404,-73.98933,0.602536
831211,Commonwealth Brewing Co.,36487245,40.758046,-73.979444,40.76404,-73.98933,0.662229
831212,Chelsea Brewing Co.,36487245,40.745682,-74.008722,40.76404,-73.98933,1.623381
831213,Carnegie Hill Brewing Co.,36487245,40.781370,-73.952445,40.76404,-73.98933,2.269684


#### The next two cells will create new table in Postgres etlproject_db and transfer cross-joined data which will contain the distances needed for the next step to be completed in Postgres using a SQL query

In [46]:
# Establish connection with Postgres etlproj_db and engine for query purposes
from postgreslogin import postgres_key

MyPostLogin = postgres_key
rds_connection_string = MyPostLogin
engine = create_engine(f'postgresql://{rds_connection_string}')

In [47]:
crossjoin_df4.to_sql(name='distance', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from distance', con=engine).head()

,brewery,airbnb_id,brew_lat,brew_lon,air_lat,air_lon,dist
0,Typhoon Brewery,2539,40.760342,-73.974326,40.64749,-73.97237,7.792564
1,The Riverosa Company,2539,40.779579,-73.977562,40.64749,-73.97237,9.124155
2,Spring Street Brewing Company,2539,40.734271,-73.992162,40.64749,-73.97237,6.080756
3,Rheingold Brewing Company,2539,40.755123,-73.985463,40.64749,-73.97237,7.463109
4,Premier Beverages LLC,2539,40.762110,-73.973069,40.64749,-73.97237,7.914048


In [50]:
session.close()

##### The following code was performed in Postgress to allow for the next steps below.

create table rank_airbnb1 as
select airbnb_id, brewery, dist, rank() over w from
distance window w as (partition by brewery order by dist asc);

-- create a new table from the query below to only include the top 5 airbnb rankings for each brewery
create table rank_airbnb2 as
select airbnb_id, brewery, dist, rank from rank_airbnb1 where rank < 6;

select * from rank_airbnb2;

-- join tables to get the airbnb NAME that matches the airbnb_id
create table airbnb_rank1 as
select rank_airbnb2.brewery, rank_airbnb2.airbnb_id, 
ny_abnb.airbnb_name, rank_airbnb2.rank, ny_abnb.price, 
rank_airbnb2.dist, (rank_airbnb2.dist * 5280) as feet_dist
from rank_airbnb2 
inner join ny_abnb on rank_airbnb2.airbnb_id = ny_abnb.airbnb_id
where rank_airbnb2.rank = 1
group by brewery, rank_airbnb2.dist, rank_airbnb2.rank, rank_airbnb2.airbnb_id, 
ny_abnb.airbnb_id, ny_abnb.airbnb_name, ny_abnb.price
order by rank_airbnb2.brewery

select * from airbnb_rank1

In [ ]:
############################  Additional code - Tuesday 11/12 ############################

#### Importing dependencies 

In [52]:
# import dependencies
import sqlalchemy
from sqlalchemy import Column, Integer, String, Numeric, Float, func
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import pandas as pd
import numpy as np
import requests, time, json
import googlemaps, gmaps
from API_KEY import API_KEY
from pprint import pprint

#### Transfer all data from the "airbnb_rank1" table in Postgres to dataframe air_rank1_df. This table contains only the 17 Breweries on our tour and the number one rank  Airbnb which is closest to the Brewery on the tour.

In [61]:
from postgreslogin import postgres_key

MyPostLogin = postgres_key
rds_connection_string = MyPostLogin
engine = create_engine(f'postgresql://{rds_connection_string}')
air_rank1_df = pd.read_sql_query('select * from airbnb_rank1', con=engine)

In [62]:
air_rank1_df

,brewery,airbnb_id,airbnb_name,rank,price,dist,feet_dist
0,Adirondack Pub & Brewery,32714651,Gorgeous Modern Cozy Manhattan Apartment!,1,120,0.010737,56.691499
1,Carnegie Hill Brewing Co.,2967631,Upper East Side 1BR,1,175,0.008716,46.022939
2,Chelsea Brewing Co.,34415580,Chelsea one bedroom with private terrace,1,280,0.025845,136.459728
3,Commonwealth Brewing Co.,19117322,Luxurious studio,1,250,0.020356,107.477510
4,Hansens Times Square Brewery,34453823,Comfort of a home!! Quiet Suite in Times Square,1,100,0.033903,179.007889
5,Harlem Brewing Co.,21272115,Visit the Big Apple! Mini-MOMA. Enjoy all of NYC!,1,75,0.013393,70.716125
6,Heartland Brewery,31122836,Luxury Union Square Loft (3000 sq ft),1,200,0.038289,202.167500
7,John Street Bar & Grill,23608594,Private Room in the Heart of Financial District,1,110,0.026261,138.656703
8,Manhattan Brewing Co. Restaurant,34485285,"Queen modern room, your sanctuary in SoHo",1,100,0.014104,74.467971
9,Nacho Mama's Brewery,25972102,Beautiful Bright Room in Soho,1,100,0.015235,80.438457


#### MERGE Number one ranked Airbnb "air_rank1_df" dataframe with the CROSS JOINED "crossjoin_df4" dataframe and sort on brewery name

In [172]:
# MERGE Number one ranked Airbnb dataframe with the CROSS JOINED dataframe and 
# sort on brewery name from the brewery dataframe
air_rank1_dftest = pd.merge(air_rank1_df, crossjoin_df4, on='airbnb_id')
air_rank1_dftest.sort_values(by='brewery_x', ascending=0)


,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
288,Typhoon Brewery,4777745,"600 SF appartment in NYC, Manhattan",1,235,0.009270,48.945640,Adirondack Pub & Brewery,40.783210,-73.979158,40.76022,-73.97440,1.606712
280,Typhoon Brewery,4777745,"600 SF appartment in NYC, Manhattan",1,235,0.009270,48.945640,Manhattan Brewing Co. Restaurant,40.723278,-74.003745,40.76022,-73.97440,2.977015
272,Typhoon Brewery,4777745,"600 SF appartment in NYC, Manhattan",1,235,0.009270,48.945640,Typhoon Brewery,40.760342,-73.974326,40.76022,-73.97440,0.009270
273,Typhoon Brewery,4777745,"600 SF appartment in NYC, Manhattan",1,235,0.009270,48.945640,The Riverosa Company,40.779579,-73.977562,40.76022,-73.97440,1.346828
274,Typhoon Brewery,4777745,"600 SF appartment in NYC, Manhattan",1,235,0.009270,48.945640,Spring Street Brewing Company,40.734271,-73.992162,40.76022,-73.97440,2.018235
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,Adirondack Pub & Brewery,32714651,Gorgeous Modern Cozy Manhattan Apartment!,1,120,0.010737,56.691499,Commonwealth Brewing Co.,40.758046,-73.979444,40.78334,-73.97927,1.746440
14,Adirondack Pub & Brewery,32714651,Gorgeous Modern Cozy Manhattan Apartment!,1,120,0.010737,56.691499,Chelsea Brewing Co.,40.745682,-74.008722,40.78334,-73.97927,3.022053
15,Adirondack Pub & Brewery,32714651,Gorgeous Modern Cozy Manhattan Apartment!,1,120,0.010737,56.691499,Carnegie Hill Brewing Co.,40.781370,-73.952445,40.78334,-73.97927,1.408994
16,Adirondack Pub & Brewery,32714651,Gorgeous Modern Cozy Manhattan Apartment!,1,120,0.010737,56.691499,Adirondack Pub & Brewery,40.783210,-73.979158,40.78334,-73.97927,0.010737


In [252]:
# following TEST code extracts the row from the MERGED DataFrame which identifies the correct data for the "Typhoon Brewery"
rank1_w_geocords = air_rank1_dftest.loc[(air_rank1_dftest["brewery_x"] == "Typhoon Brewery") & (
    air_rank1_dftest["rank"] == 1) & (air_rank1_dftest["brewery_y"] == "Typhoon Brewery"), :]
rank1_w_geocords

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
272,Typhoon Brewery,4777745,"600 SF appartment in NYC, Manhattan",1,235,0.00927,48.94564,Typhoon Brewery,40.760342,-73.974326,40.76022,-73.9744,0.00927


In [ ]:
#############################################################################################################

#### The following 17 cells extract the specific row required for each brewery on the tour and the corresponding AirBnB that is closest to that brewery, the distance in miles and feet and the geo coordinates necessary to create a heat map for both the breweries and the AirBnB's on the tour.

In [210]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: 'Typhoon Brewery' in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: 'Typhoon Brewery' in brewery_y)]
air_rank1_df01 = air_rank1_dfb
air_rank1_df01

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
272,Typhoon Brewery,4777745,"600 SF appartment in NYC, Manhattan",1,235,0.00927,48.94564,Typhoon Brewery,40.760342,-73.974326,40.76022,-73.9744,0.00927


In [186]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: 'Adirondack Pub & Brewery' in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: 'Adirondack Pub & Brewery' in brewery_y)]
air_rank1_df02 = air_rank1_dfb
air_rank1_df02

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
16,Adirondack Pub & Brewery,32714651,Gorgeous Modern Cozy Manhattan Apartment!,1,120,0.010737,56.691499,Adirondack Pub & Brewery,40.78321,-73.979158,40.78334,-73.97927,0.010737


In [187]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: 'Carnegie Hill Brewing Co.' in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: 'Carnegie Hill Brewing Co.' in brewery_y)]
air_rank1_df03 = air_rank1_dfb
air_rank1_df03

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
32,Carnegie Hill Brewing Co.,2967631,Upper East Side 1BR,1,175,0.008716,46.022939,Carnegie Hill Brewing Co.,40.78137,-73.952445,40.78139,-73.95261,0.008716


In [188]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: 'Chelsea Brewing Co.' in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: 'Chelsea Brewing Co.' in brewery_y)]
air_rank1_df04 = air_rank1_dfb
air_rank1_df04

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
48,Chelsea Brewing Co.,34415580,Chelsea one bedroom with private terrace,1,280,0.025845,136.459728,Chelsea Brewing Co.,40.745682,-74.008722,40.74557,-74.00825,0.025845


In [189]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: 'Commonwealth Brewing Co.' in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: 'Commonwealth Brewing Co.' in brewery_y)]
air_rank1_df05 = air_rank1_dfb
air_rank1_df05

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
64,Commonwealth Brewing Co.,19117322,Luxurious studio,1,250,0.020356,107.47751,Commonwealth Brewing Co.,40.758046,-73.979444,40.758,-73.97906,0.020356


In [190]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: 'Hansens Times Square Brewery' in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: 'Hansens Times Square Brewery' in brewery_y)]
air_rank1_df06 = air_rank1_dfb
air_rank1_df06

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
80,Hansens Times Square Brewery,34453823,Comfort of a home!! Quiet Suite in Times Square,1,100,0.033903,179.007889,Hansens Times Square Brewery,40.755584,-73.986483,40.75588,-73.987,0.033903


In [191]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: 'Harlem Brewing Co.' in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: 'Harlem Brewing Co.' in brewery_y)]
air_rank1_df07 = air_rank1_dfb
air_rank1_df07

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
96,Harlem Brewing Co.,21272115,Visit the Big Apple! Mini-MOMA. Enjoy all of NYC!,1,75,0.013393,70.716125,Harlem Brewing Co.,40.806607,-73.953556,40.80674,-73.95337,0.013393


In [193]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: 'Heartland Brewery' in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: 'Heartland Brewery' in brewery_y)]
air_rank1_df08 = air_rank1_dfb
air_rank1_df08

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
112,Heartland Brewery,31122836,Luxury Union Square Loft (3000 sq ft),1,200,0.038289,202.1675,Heartland Brewery,40.73689,-73.990792,40.73732,-73.99033,0.038289


In [194]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: 'John Street Bar & Grill' in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: 'John Street Bar & Grill' in brewery_y)]
air_rank1_df09 = air_rank1_dfb
air_rank1_df09

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
128,John Street Bar & Grill,23608594,Private Room in the Heart of Financial District,1,110,0.026261,138.656703,John Street Bar & Grill,40.70997,-74.008741,40.71012,-74.00828,0.026261


In [195]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: 'Manhattan Brewing Co. Restaurant' in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: 'Manhattan Brewing Co. Restaurant' in brewery_y)]
air_rank1_df10 = air_rank1_dfb
air_rank1_df10

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
144,Manhattan Brewing Co. Restaurant,34485285,"Queen modern room, your sanctuary in SoHo",1,100,0.014104,74.467971,Manhattan Brewing Co. Restaurant,40.723278,-74.003745,40.72324,-74.00401,0.014104


In [197]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: "Nacho Mama's Brewery" in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: "Nacho Mama's Brewery" in brewery_y)]
air_rank1_df11 = air_rank1_dfb
air_rank1_df11

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
160,Nacho Mama's Brewery,25972102,Beautiful Bright Room in Soho,1,100,0.015235,80.438457,Nacho Mama's Brewery,40.723304,-74.003707,40.72352,-74.00365,0.015235


In [198]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: "Neptune Brewery" in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: "Neptune Brewery" in brewery_y)]
air_rank1_df12 = air_rank1_dfb
air_rank1_df12

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
176,Neptune Brewery,9175459,"Location, Location, Location Meat Packing Dist...",1,180,0.037588,198.465062,Neptune Brewery,40.742893,-74.006416,40.74236,-74.00656,0.037588


In [199]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: "New Amsterdam Brewing Co." in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: "New Amsterdam Brewing Co." in brewery_y)]
air_rank1_df13 = air_rank1_dfb
air_rank1_df13

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
192,New Amsterdam Brewing Co.,16243676,APT FOR UP TO 4! Dream location & private terr...,1,250,0.007378,38.958397,New Amsterdam Brewing Co.,40.738623,-73.987135,40.73873,-73.98714,0.007378


In [200]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: "Premier Beverages LLC" in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: "Premier Beverages LLC" in brewery_y)]
air_rank1_df14 = air_rank1_dfb
air_rank1_df14

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
208,Premier Beverages LLC,5262333,Excellent Midtown Location!,1,149,0.01944,102.644826,Premier Beverages LLC,40.76211,-73.973069,40.76186,-73.97324,0.01944


In [201]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: "Rheingold Brewing Company" in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: "Rheingold Brewing Company" in brewery_y)]
air_rank1_df15 = air_rank1_dfb
air_rank1_df15

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
224,Rheingold Brewing Company,34485739,Midtown Manhattan Private Alcove Suite,1,100,0.022789,120.327847,Rheingold Brewing Company,40.755123,-73.985463,40.75481,-73.9856,0.022789


In [202]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: "Spring Street Brewing Company" in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: "Spring Street Brewing Company" in brewery_y)]
air_rank1_df16 = air_rank1_dfb
air_rank1_df16

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
240,Spring Street Brewing Company,11198278,2BR WITH PRIVATE PATIO EAST VILLAGE,1,245,0.030513,161.108146,Spring Street Brewing Company,40.734271,-73.992162,40.73455,-73.99171,0.030513


In [203]:
air_rank1_dfa = air_rank1_df2[air_rank1_df2['brewery_x'].map(lambda brewery_x: "The Riverosa Company" in brewery_x)]
air_rank1_dfb = air_rank1_dfa[air_rank1_dfa['brewery_y'].map(lambda brewery_y: "The Riverosa Company" in brewery_y)]
air_rank1_df17 = air_rank1_dfb
air_rank1_df17

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
256,The Riverosa Company,29957848,Beautiful UWS Apt. \\ 4 mins from Central Park,1,200,0.006614,34.924451,The Riverosa Company,40.779579,-73.977562,40.77951,-73.97765,0.006614


#### Now join the 17 results above into one dataframe below

In [221]:
# join the 17 results above into one dataframe
brewsjoined_df = pd.concat([air_rank1_df01,air_rank1_df02,air_rank1_df03,air_rank1_df04,air_rank1_df05,\
        air_rank1_df06,air_rank1_df07,air_rank1_df08,air_rank1_df09,air_rank1_df10,air_rank1_df11,\
        air_rank1_df12,air_rank1_df13,air_rank1_df14,air_rank1_df15,air_rank1_df16,air_rank1_df17],\
        ignore_index=True)

In [222]:
brewsjoined_df.sort_values('brewery_x')
brewsjoined_df

,brewery_x,airbnb_id,airbnb_name,rank,price,dist_x,feet_dist,brewery_y,brew_lat,brew_lon,air_lat,air_lon,dist_y
0,Typhoon Brewery,4777745,"600 SF appartment in NYC, Manhattan",1,235,0.009270,48.945640,Typhoon Brewery,40.760342,-73.974326,40.76022,-73.97440,0.009270
1,Adirondack Pub & Brewery,32714651,Gorgeous Modern Cozy Manhattan Apartment!,1,120,0.010737,56.691499,Adirondack Pub & Brewery,40.783210,-73.979158,40.78334,-73.97927,0.010737
2,Carnegie Hill Brewing Co.,2967631,Upper East Side 1BR,1,175,0.008716,46.022939,Carnegie Hill Brewing Co.,40.781370,-73.952445,40.78139,-73.95261,0.008716
3,Chelsea Brewing Co.,34415580,Chelsea one bedroom with private terrace,1,280,0.025845,136.459728,Chelsea Brewing Co.,40.745682,-74.008722,40.74557,-74.00825,0.025845
4,Commonwealth Brewing Co.,19117322,Luxurious studio,1,250,0.020356,107.477510,Commonwealth Brewing Co.,40.758046,-73.979444,40.75800,-73.97906,0.020356
5,Hansens Times Square Brewery,34453823,Comfort of a home!! Quiet Suite in Times Square,1,100,0.033903,179.007889,Hansens Times Square Brewery,40.755584,-73.986483,40.75588,-73.98700,0.033903
6,Harlem Brewing Co.,21272115,Visit the Big Apple! Mini-MOMA. Enjoy all of NYC!,1,75,0.013393,70.716125,Harlem Brewing Co.,40.806607,-73.953556,40.80674,-73.95337,0.013393
7,Heartland Brewery,31122836,Luxury Union Square Loft (3000 sq ft),1,200,0.038289,202.167500,Heartland Brewery,40.736890,-73.990792,40.73732,-73.99033,0.038289
8,John Street Bar & Grill,23608594,Private Room in the Heart of Financial District,1,110,0.026261,138.656703,John Street Bar & Grill,40.709970,-74.008741,40.71012,-74.00828,0.026261
9,Manhattan Brewing Co. Restaurant,34485285,"Queen modern room, your sanctuary in SoHo",1,100,0.014104,74.467971,Manhattan Brewing Co. Restaurant,40.723278,-74.003745,40.72324,-74.00401,0.014104


In [223]:
# clean up the headers for joined list of Number 1 Ranked Airbnb's and Breweries on the tour and drop unnecessary columns
brewsjoined_df.columns = ['brewery','airbnb_id', 'airbnb_name', 'rank', 'price', 'dist_miles', 'dist_feet', \
                          'brewery_y', 'brew_lat','brew_lon','air_lat','air_lon', 'dist_miles2']
brewsjoined_df2 = brewsjoined_df.drop(['brewery_y', 'dist_miles2'], axis=1)
brewsjoined_df2

,brewery,airbnb_id,airbnb_name,rank,price,dist_miles,dist_feet,brew_lat,brew_lon,air_lat,air_lon
0,Typhoon Brewery,4777745,"600 SF appartment in NYC, Manhattan",1,235,0.009270,48.945640,40.760342,-73.974326,40.76022,-73.97440
1,Adirondack Pub & Brewery,32714651,Gorgeous Modern Cozy Manhattan Apartment!,1,120,0.010737,56.691499,40.783210,-73.979158,40.78334,-73.97927
2,Carnegie Hill Brewing Co.,2967631,Upper East Side 1BR,1,175,0.008716,46.022939,40.781370,-73.952445,40.78139,-73.95261
3,Chelsea Brewing Co.,34415580,Chelsea one bedroom with private terrace,1,280,0.025845,136.459728,40.745682,-74.008722,40.74557,-74.00825
4,Commonwealth Brewing Co.,19117322,Luxurious studio,1,250,0.020356,107.477510,40.758046,-73.979444,40.75800,-73.97906
5,Hansens Times Square Brewery,34453823,Comfort of a home!! Quiet Suite in Times Square,1,100,0.033903,179.007889,40.755584,-73.986483,40.75588,-73.98700
6,Harlem Brewing Co.,21272115,Visit the Big Apple! Mini-MOMA. Enjoy all of NYC!,1,75,0.013393,70.716125,40.806607,-73.953556,40.80674,-73.95337
7,Heartland Brewery,31122836,Luxury Union Square Loft (3000 sq ft),1,200,0.038289,202.167500,40.736890,-73.990792,40.73732,-73.99033
8,John Street Bar & Grill,23608594,Private Room in the Heart of Financial District,1,110,0.026261,138.656703,40.709970,-74.008741,40.71012,-74.00828
9,Manhattan Brewing Co. Restaurant,34485285,"Queen modern room, your sanctuary in SoHo",1,100,0.014104,74.467971,40.723278,-74.003745,40.72324,-74.00401


In [232]:
# create a list of the brewery latitudes
brew_lat_list = brewsjoined_df2["brew_lat"].tolist()
brew_lat_list

[40.760342,
 40.7832098,
 40.7813704,
 40.7456816,
 40.7580462,
 40.755584,
 40.8066068,
 40.73688990000001,
 40.7099702,
 40.7232777,
 40.7233037,
 40.74289340000001,
 40.7386232,
 40.76211,
 40.7551234,
 40.7342711,
 40.7795789]

In [233]:
# create a list of the brewery longitudes
brew_lon_list = brewsjoined_df2["brew_lon"].tolist()
brew_lon_list

[-73.974326,
 -73.9791577,
 -73.9524453,
 -74.0087216,
 -73.97944439999999,
 -73.98648279999999,
 -73.9535563,
 -73.990792,
 -74.0087412,
 -74.0037451,
 -74.00370749999999,
 -74.0064163,
 -73.9871351,
 -73.973069,
 -73.98546329999999,
 -73.9921624,
 -73.9775621]

In [245]:
# create a list of the airbnb latitudes
air_lat_list = brewsjoined_df2["air_lat"].tolist()
air_lat_list

[40.760220000000004,
 40.78334,
 40.78139,
 40.74557,
 40.758,
 40.75588,
 40.806740000000005,
 40.737320000000004,
 40.71012,
 40.723240000000004,
 40.72352,
 40.74236,
 40.73873,
 40.76186,
 40.75481,
 40.73455,
 40.779509999999995]

In [246]:
# create a list of the airbnb longitudes
air_lon_list = brewsjoined_df2["air_lon"].tolist()
air_lon_list

[-73.9744,
 -73.97927,
 -73.95260999999999,
 -74.00825,
 -73.97906,
 -73.987,
 -73.95336999999999,
 -73.99033,
 -74.00828,
 -74.00401,
 -74.00365,
 -74.00656,
 -73.98714,
 -73.97324,
 -73.9856,
 -73.99171,
 -73.97765]

#### Installed gmplot and then import below to support creating heat maps (pip install gmplot in Bash)

In [225]:
import gmplot

In [254]:
# create a heat map for the list of breweries on the tour
latitude_list = brew_lat_list 
longitude_list = brew_lon_list

brew_map = gmplot.GoogleMapPlotter(40.7127837, -74.0059413, 12)

# heatmap plot heating Type 
# points on the Google map 
brew_map.heatmap( latitude_list, longitude_list ) 
  
brew_map.draw( "C:\\Users\\Owner\\GitHub\\ETL-project\\brew_map.html" ) 

In [251]:
# create a heat map for the list of airbnb's that are closest to the breweries on the tour
air_latitude_list = air_lat_list 
air_longitude_list = air_lon_list

airbnb_map = gmplot.GoogleMapPlotter(40.7127837, -74.0059413, 12)

# heatmap plot heating Type 
# points on the Google map 
airbnb_map.heatmap( air_latitude_list, air_longitude_list ) 
  
airbnb_map.draw( "C:\\Users\\Owner\\GitHub\\ETL-project\\airbnb_map.html" ) 

#### Finally: Sort, Set Index to Brewery and transfer from dataframe "brewsjoined_df2" into Postgres

In [261]:
brewsjoined_df3 = brewsjoined_df2.sort_values(by= 'brewery')
brewsjoined_df3

,brewery,airbnb_id,airbnb_name,rank,price,dist_miles,dist_feet,brew_lat,brew_lon,air_lat,air_lon
1,Adirondack Pub & Brewery,32714651,Gorgeous Modern Cozy Manhattan Apartment!,1,120,0.010737,56.691499,40.783210,-73.979158,40.78334,-73.97927
2,Carnegie Hill Brewing Co.,2967631,Upper East Side 1BR,1,175,0.008716,46.022939,40.781370,-73.952445,40.78139,-73.95261
3,Chelsea Brewing Co.,34415580,Chelsea one bedroom with private terrace,1,280,0.025845,136.459728,40.745682,-74.008722,40.74557,-74.00825
4,Commonwealth Brewing Co.,19117322,Luxurious studio,1,250,0.020356,107.477510,40.758046,-73.979444,40.75800,-73.97906
5,Hansens Times Square Brewery,34453823,Comfort of a home!! Quiet Suite in Times Square,1,100,0.033903,179.007889,40.755584,-73.986483,40.75588,-73.98700
6,Harlem Brewing Co.,21272115,Visit the Big Apple! Mini-MOMA. Enjoy all of NYC!,1,75,0.013393,70.716125,40.806607,-73.953556,40.80674,-73.95337
7,Heartland Brewery,31122836,Luxury Union Square Loft (3000 sq ft),1,200,0.038289,202.167500,40.736890,-73.990792,40.73732,-73.99033
8,John Street Bar & Grill,23608594,Private Room in the Heart of Financial District,1,110,0.026261,138.656703,40.709970,-74.008741,40.71012,-74.00828
9,Manhattan Brewing Co. Restaurant,34485285,"Queen modern room, your sanctuary in SoHo",1,100,0.014104,74.467971,40.723278,-74.003745,40.72324,-74.00401
10,Nacho Mama's Brewery,25972102,Beautiful Bright Room in Soho,1,100,0.015235,80.438457,40.723304,-74.003707,40.72352,-74.00365


In [264]:
# Establish connection with Postgres etlproj_db and engine for query purposes
from postgreslogin import postgres_key

MyPostLogin = postgres_key
rds_connection_string = MyPostLogin
engine = create_engine(f'postgresql://{rds_connection_string}')
brewsjoined_df3.to_sql(name='brewery_tour_list', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from brewery_tour_list', con=engine).head()

,brewery,airbnb_id,airbnb_name,rank,price,dist_miles,dist_feet,brew_lat,brew_lon,air_lat,air_lon
0,Adirondack Pub & Brewery,32714651,Gorgeous Modern Cozy Manhattan Apartment!,1,120,0.010737,56.691499,40.783210,-73.979158,40.78334,-73.97927
1,Carnegie Hill Brewing Co.,2967631,Upper East Side 1BR,1,175,0.008716,46.022939,40.781370,-73.952445,40.78139,-73.95261
2,Chelsea Brewing Co.,34415580,Chelsea one bedroom with private terrace,1,280,0.025845,136.459728,40.745682,-74.008722,40.74557,-74.00825
3,Commonwealth Brewing Co.,19117322,Luxurious studio,1,250,0.020356,107.477510,40.758046,-73.979444,40.75800,-73.97906
4,Hansens Times Square Brewery,34453823,Comfort of a home!! Quiet Suite in Times Square,1,100,0.033903,179.007889,40.755584,-73.986483,40.75588,-73.98700


In [ ]:
session.close()